In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/taindp98/endoscopic_classification.git

In [3]:
!pip install einops
!pip install timm
!pip install grad-cam

In [4]:
import numpy as np
import time
import torch
import json
import os
import pandas as pd
import warnings
from glob import glob
import cv2
from PIL import Image
from fastprogress import progress_bar, master_bar
from matplotlib import pyplot as plt
from datetime import datetime,date

import torchvision
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torch import nn
from torchvision import datasets, transforms
from torch.nn import DataParallel
import torch.nn.functional as F
# from timm.scheduler.step_lr import StepLRScheduler
from torch.optim.lr_scheduler import StepLR

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.utils import class_weight

from yacs.config import CfgNode as CN

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('always') 

In [6]:
import sys
# print(sys.path)
if './endoscopic_classification/code' not in sys.path:
    sys.path.append('./endoscopic_classification/code')

In [7]:
from utils import train_one, eval_one, show_batch, save_checkpoint, load_checkpoint
from dataset import GIDataset
from build import build_model
from configs import config
from opts import build_optimizer
from loss import choose_loss_fnc

In [8]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [9]:
_KVASIR_CAPSULE = '../input/endoscopy/kvasir_capsule_deform'
_RESOURCE = '../input/endoscopy/fold_0.csv'
# _CHECKPOINT = '../input/checkpoint-endoscopy/coat_03.pth'
# _KVASIR_CAPSULE = 'D:\\data\\endoscopic\\kvasir_capsule_deform'
# _RESOURCE = '../resource/kvasir_capsule/df_train_valid_balance.csv'

In [10]:
config.DATA.BATCH_SIZE = 32
config.DATA.IMG_SIZE = 224
config.MODEL.NUM_CLASSES = 11
config.MODEL.TYPE = 'attguide'
config.MODEL.MARGIN = True
config.DATA_PATH = _KVASIR_CAPSULE
config.TRAIN.DEVICE = str(device)
config.TRAIN.USE_CHECKPOINT = False
config.TRAIN.CLASSES_WEIGHT = True
config.TRAIN.IS_TRAIN = True
config.TRAIN._CHECKPOINT_MODEL = None
config.TRAIN._CHECKPOINT_HEAD = None
config.TRAIN.OPTIMIZER.NAME = 'sgd'

In [11]:
df_anno = pd.read_csv(_RESOURCE)
df_anno['path'] = [os.path.join(config.DATA_PATH,item) for item in list(df_anno['path'])]
df_train = df_anno[df_anno['is_valid']==False]
df_valid = df_anno[df_anno['is_valid']==True]

In [12]:
model = build_model(config)

In [13]:
train_transform = transforms.Compose([ 
    transforms.Resize(config.DATA.IMG_SIZE),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.RandomAffine(
                 degrees=30,  
                 interpolation=Image.NEAREST),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(config.DATA.IMG_SIZE, scale=(0.63, 1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

test_transform = transforms.Compose([ 
    transforms.Resize(config.DATA.IMG_SIZE),
    transforms.CenterCrop(config.DATA.IMG_SIZE),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])


train_ds = GIDataset(df_train, config, train_transform) 
valid_ds = GIDataset(df_valid, config, test_transform) 

train_dl = torch.utils.data.DataLoader(
                                    dataset = train_ds, 
                                    batch_size = config.DATA.BATCH_SIZE, 
                                    shuffle = True, 
                                    num_workers = config.DATA.NUM_WORKERS)
valid_dl = torch.utils.data.DataLoader( 
                                    dataset = valid_ds, 
                                    batch_size = config.DATA.BATCH_SIZE, 
                                    shuffle = False, 
                                    num_workers = config.DATA.NUM_WORKERS)

In [14]:
inputs, classes = next(iter(train_dl))
inputs, classes = inputs[:8,:,:], classes[:8]
out = torchvision.utils.make_grid(inputs)
show_batch(out, title=classes)

In [15]:
class_weights = class_weight.compute_class_weight(class_weight  = 'balanced',
                                              classes  = np.unique(df_train['target']).tolist(),
                                              y = list(df_train['target']));
class_weights = torch.tensor(class_weights,dtype=torch.float).to(device);
criterion = choose_loss_fnc(config, class_weights)
criterion = criterion.to(device);

In [16]:
optimizer = build_optimizer(config, model)

In [17]:
lr_scheduler = StepLR(optimizer,  
                      step_size = config.TRAIN.LR_SCHEDULER.STEP_SIZE, 
                      gamma = config.TRAIN.LR_SCHEDULER.DECAY_RATE)

In [ ]:
for epoch in range(1, config.TRAIN.EPOCHS + 1):
    train_loss = train_one(train_dl, model, criterion, optimizer, config, device)
    print(f'Training epoch: {epoch}; Learning rate: {optimizer.param_groups[0]["lr"]}')
    print(f'\tTrain Loss: {train_loss.avg:.3f}')
    lr_scheduler.step()
    if (epoch)%2==0:
        valid_loss, valid_metric = eval_one(valid_dl, model, criterion, config, device)
        print(f'\tValid Loss: {valid_loss.avg:.3f}')
        print(f'\tMetric: {valid_metric}')